In [1]:
# modified from https://github.com/pytorch/examples/blob/master/imagenet/main.py
import argparse
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import models

from qconv import QConv2d
from qlinear import QLinear

/home/usami/miniconda3/envs/human_action/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/usami/miniconda3/envs/human_action/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c107WarningC1ENS_7variantIJNS0_11UserWarningENS0_18DeprecationWarningEEEERKNS_14SourceLocationENSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEEb'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
#imports
import pandas as pd
import numpy as np
# from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

df_unsw_normal = pd.read_csv('/home/usami/Protocol-Based-Deep-Intrusion-Detection-for-DoS-Normal-and-DDoS-Attacks/normal_ddos_dos_classification/dataset/train/unsw_normal.csv')
df_bot_dos = pd.read_csv('/home/usami/Protocol-Based-Deep-Intrusion-Detection-for-DoS-Normal-and-DDoS-Attacks/normal_ddos_dos_classification/dataset/train/bot_iot_dos.csv')
df_bot_ddos = pd.read_csv('/home/usami/Protocol-Based-Deep-Intrusion-Detection-for-DoS-Normal-and-DDoS-Attacks/normal_ddos_dos_classification/dataset/train/bot_iot_ddos.csv')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
df_combined = pd.concat([df_unsw_normal, df_bot_dos, df_bot_ddos], ignore_index=True)

x = df_combined.loc[:, df_combined.columns != 'category']
# x = (x-x.min())/(x.max()-x.min())

df_combined.category = pd.factorize(df_combined.category)[0]
y = df_combined['category']

scaler = StandardScaler()
x = scaler.fit_transform(x)
for i in [0,1,2,3,7,8]:
    x[:,i] = np.where(x[:,i] > 0, np.log(x[:,i]), 0)
    
x_label_data = torch.tensor(x, dtype=torch.float).to(device)
y_label_data = torch.tensor(y, dtype=torch.long).to(device)

dataset = TensorDataset(x_label_data, y_label_data)
train_size = int(0.7 * len(dataset))
valid_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - valid_size
train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size])

# Create data loaders
batch_size = 8192
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

/tmp/ipykernel_1202194/4249798583.py:12: RuntimeWarning: invalid value encountered in log
  x[:,i] = np.where(x[:,i] > 0, np.log(x[:,i]), 0)


In [ ]:
class binary_weight(torch.autograd.Function):
    @staticmethod
    def forward(ctx, weight_f):
        scale = weight_f.abs().sum(dim=list(range(1, weight_f.ndim)), keepdim=True) / weight_f[0].numel()
        weight_b = weight_f.sign() * scale
        return weight_b

    @staticmethod
    def backward(ctx, *grad_outputs):
        return grad_outputs


class BinaryWeight(nn.Module):
    def __init__(self, config: dict, *args, **kwargs):
        super().__init__()
        self.config = config

    def forward(self, weight_f):
        weight_b = binary_weight.apply(weight_f)
        return weight_b


In [4]:
class Net_tea(nn.Module):
    def __init__(self):
        super(Net_tea, self).__init__()

        self.linear1 = nn.Linear(13, 128)
        self.bn1 = nn.BatchNorm1d(128)
        # self.i1 = nn.Identity()

        self.linear2 = QLinear(128, 128, bias=False)
        self.bn2 = nn.BatchNorm1d(128)
        # self.i2 = nn.Identity()

        self.linear3 = QLinear(128, 128, bias=False)
        self.bn3 = nn.BatchNorm1d(128)
        # self.i3 = nn.Identity()

        self.linear4 = nn.Linear(128, 3)
        self.act4 = nn.Softmax(dim = 1)

    def forward(self, x):
        # x = self.net(x)
        x = self.linear1(x)
        x = self.bn1(x)
        # x = self.i1(x)

        x = self.linear2(x)
        x = self.bn2(x)
        # x = self.i2(x)

        x = self.linear3(x)
        x = self.bn3(x)
        # x = self.i3(x)

        x = self.linear4(x)
        x = self.act4(x)
        return x
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ter1 = Net_tea().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_ter1.parameters(), 0.01,
                                momentum=0.9,
                                weight_decay=1e-4)
num_epochs = 2
for epoch in range(num_epochs):
    model_ter1.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_ter1(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # model_ter1.set_weights_and_biases()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}', end='\r')
    
    model_ter1.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model_ter1(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Accuracy: {accuracy:.2f}%')

# Final evaluation on the test set
model_ter1.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_ter1(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')

torch.save(model_ter1.state_dict(), 'saved_best_model_ter.pt')

Epoch [1/2], Validation Accuracy: 91.70%7
Epoch [2/2], Validation Accuracy: 91.86%3
Test Accuracy: 91.87%
